# Noisy Neural Networks
The penultimate improvement of the DQN agent [1] that is still missing in the agent are "Noisy Networks", which were published by Fortunato et al., (2017) [2]. The idea behind Noisy Nets is to add noise of learnable magnitude to the weights of the neural network as a mechanism to drive exploration instead of using an $\epsilon$-greedy policy.

The weights $\theta$ of a noisy linear layer are given by [2]: 
$$ \theta = \mu + \Sigma \odot \epsilon$$  
where $\mu$ corresponds to the weights of a regular linear layer. $\Sigma$ is the magnitude of the noise term which is learned in the optimization step together with $\mu$. $\epsilon$ is a matrix with values drawn from some random distribution. New values for $\epsilon$ are sampled at different times in the algorithm (see below). $\theta$ and $\epsilon$ have the same dimensions as $\theta$ and $\odot$ is the element-wise multiplication. The bias term can be constructed in a similar way.

## Implementation
The implementation loosely follows the noisy networks implementation from the Deep Reinforcement Learnng: Hands-on book [3], however, instead of sampling new noise on each call of the neural network I implemented a reset_noise method. Calling this method allows more fine grained control of when noise is sampled in the agent. 

Noise is not sampled exactly as in the original manuscript where three different noise terms for the q-net (one for action selection, one for action selection in the training step and one for calculating the Q-value) and one noise term for the target network are used (see C.1 in [2]).  
I chose to only resample noise for the q-network before action selection in the training step and before calculating the q-value estimates. Therefore I use the second noise sample also for action selection when collecting experience. Noise for the target network is sampled before the optimization step.

Furtonato et al. (2017) suggest that $\sigma_0$ (the initial noise level of the noisy network) can be set to $0.5/\sqrt p$ (where p is the input dimension) and does not need to be tuned as a hyperparameter. Therefore, I did not include $\sigma_0$ in the settings dictionary and hardcoded this value.  
In my first attempt (run027) I accidently used values of $\sigma_0 = 1/\sqrt p$, $\alpha = 0.5$ and $\beta_0 = 0.5$. After changing the values to $\sigma_0 = 1/\sqrt p$, $\alpha = 0.6$ and $\beta_0 = 0.4$ the results were different. While not being conclusive (since I changed the PER related values $\alpha$, $\beta_0$) it might be advantageous to include $\sigma_0$ a hyperparameter and fine tune it in the future.

## Results
The radar plot as well as the barplot that compares the three agents shows that not using Noisy Networks (run024) leads to better results than the two agents with Noisy Networks (run027 and run028). In particular the Cartpole experiment and the different Scale experiments show worse performance when using Noisy Nets.

![Radar plot of one agent without Noisy Nets (run024) and two agents with noisy nets and different $\sigma0$.](./figures/noisy_radar.png)

![Bar plot comparing one agent without Noisy Nets (run024) and two agents with noisy nets and different $\sigma0$.](./figures/noisy_barplot.png)

However, the average returns on the bandit experiments (see figure below) shows that Noisy Networks show promising results on some tasks. On the bandit experiments the final score is better when using Noisy Networks, possibly due to vanishing noise that favors exploitation over exploration towards the end of the episode. Run028 seems to learn good policies faster, however, there are a few episodes of the experiment in which the agent does converge to a suboptimal solution.

![Average return on the Bandit experiments.](./figures/noisy_bandit.png)

When comparing the two noisy network agents with different values of inital noise (run027 has $\sigma_0 = 1/\sqrt p$ while run028 has $\sigma_0 = 0.5/\sqrt p$) one can observe $\sigma_0$ might influence the results.
However, none of the two runs is clearly better than the other one and I also used different values for the PER hyperparamters.
![One on one comparison of the two noisy network agents on all experiments.](./figures/noisy_sigma0_barplot.png)

## Discussion
While it is still possible that the subpar results are due to non-optimal hyperparameters, I will not investigate hyperparamaters at this point due to computational costs (I will elaborate on this more in the last update).

### Theoretical considerations
Furtonato et al. (2017) [2] say that their procedure is "automatically tuning the level of noise ... to drive down (or up) the level of noise ... as needed". 
The goal of the optimization procedure is to learn a good representation of the q-values and there is no direct incentive for the algorithm to explore. Thus, similar to some of the reviewers on OpenReview [4], I do not understand why the noise term in the Noisy Network algorithm in this form should actively drive exploration and not converge to zero.

Plappert et al., 2017 [5] discusses that using parameter noise leads to deterministic actions for one state at least between new noise samples instead of choosing a totally random action with some probability $\epsilon$ (in $\epsilon$-greedy policies). Additionally, when using parameter noise, the actions are in some neighborhood of the action without noise.

In my opinion using Noisy Networks help with the training of Reinforcement Learning agents for two reasons:
* similar to an $\epsilon$-greedy strategy the noisy networks add an element of randomness that leads to exploration, in particular in the beginning of training. Furthermore, assuming that the noise level should theoretically decay, there are similarities with $\epsilon$-greedy methods that use an $\epsilon$ decay schedule. In the case of noisy nets the optimizer's learning rate plays the role of the decay rate and the initial values of the noise term $\sigma_0$ corresponds to $\epsilon_0$.
* the parameter noise could potentially lead to a more robust neural network that generalizes better (similar to Dropout layers that stabilize training of neural networks by adding an element of randomness). I think that this is what one of the reviewers [4] mentioned by effects on the optimization.

## References

The figures here were produced by the analysis Jupyter Notebook from [the BSuite code repository](https://github.com/deepmind/bsuite) and [6].

[1] Mnih, Volodymyr, et al. Human-level control through deep reinforcement learning. Nature, 2015.  
[2] Fortunato, Meire, et al. "Noisy networks for exploration." arXiv preprint arXiv:1706.10295 2017.  
[3] Lapan, Maxim. Deep Reinforcement Learning Hands-On, Packt Publishing Ltd, 2018.  
[4] https://openreview.net/forum?id=rywHCPkAW, last accessed: 2020-04-30.  
[5] Plappert, Matthias, et al. "Parameter space noise for exploration." arXiv preprint arXiv:1706.01905, 2017.  
[6] Osband, Ian, et al. "Behaviour Suite for Reinforcement Learning." arXiv preprint arXiv:1908.03568, 2019.  
